# Imports

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import itertools
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import keras
import math
from sklearn.model_selection import KFold
import sklearn.metrics
from sklearn.model_selection import train_test_split
from operator import itemgetter
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from scipy import stats
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

# Setting Global Variables

The `segments` dictionary translates the primary behaviors to the overall segemnts

In [ ]:
# Number of epochs for neural network training
NUM_EPOCHS = 30
# Sensors for training
sensors = ['Hip', 'Wrist', 'Chest', 'Thigh']
segments = {
    "LES- socializing, communicating, leisure time not screen":"LES",
      "LES- screen based leisure time (TV, video game, computer)":"LES",
      "HA- animals and pets":"HA",
      "HA- housework":"HA",
      "HA- food prep and cleanup":"HA",
      "HA- household management/other household activities":"HA",
      "HA- interior maintenance, repair, & decoration":"HA",
      "HA- lawn, garden and houseplants":"HA",
      "HA- exterior maintenance, repair, & decoration":"HA",
      "CA- caring for and helping adults":"HA",
      "CA- caring for and helping children":"HA",
      "WRK- screen based - Education and Health Services":"WRK",
      "WRK- screen based - Office (business, professional services, finance, info)":"WRK",
      "WRK- general - Education and Health Services":"WRK",
      "WRK- general - Office (business, professional services, finance, info)":"WRK",
      "EDU- taking class, research, homework":"WRK",
      "PC- groom, health-related":"HA",
      "PC- other personal care":"HA",
      "EAT- eating and drinking, waiting":"HA",
      "PUR- purchasing goods and services":"HA",
      "ORG- organizational civic, volunteer, and religious activities":"COM",
      "EX- attending sport, recreational event, or performance":"COM",
      "EX- walking":"EX",
      "EX- jogging":"EX",
      "EX- baseball":"EX",
      "EX- other":"EX",
      "TRAV- walking":"COM",
      "TRAV- passenger bus or train":"COM",
      "TRAV- biking":"COM",
      "TRAV- driver (car/truck/motorcycle)":"COM",
      "OTHER- non codable":"OTHER",
      "None":"OTHER"
}

# Running the Pipeline

Results from running the data pipeline.

## Hip

In [ ]:
# Run single sensor neural network for detailed activity labels
activity = 'updated_final_activity'
losses = []
accs = []
preds = []
s = "Hip"

# attain dataset for running the model 
data = clean_data(s, activity, True)

# run the model and output revelant results 
acc, cnf_mat = run_nn(data, "Hip", activity, post = False)
print("---------------------------------------------")
print("Accuracy: ",acc)
print(cnf_mat)
print_confusion_matrix(cnf_mat, ['sit/lie', 'stand and move', 'walking', 'running','bicycling'])

## Chest

In [ ]:
# Run single sensor neural network for detailed activity labels
activity = 'updated_final_activity'
losses = []
accs = []
preds = []
s = "Chest"
data = clean_data(s, activity, True)
#res = run_nn(data, s, activity)
#sum_cnf_matrix = np.sum(res[2], axis = 0)

acc, cnf_mat = run_nn(data, "Chest", activity, post = False)
print("---------------------------------------------")
print("Accuracy: ",acc)
print_confusion_matrix(cnf_mat, ['sit/lie', 'stand and move', 'walking', 'running','bicycling'])

#pred = pd.DataFrame(pred)

#pred.to_csv(''+s+'_predictions.csv')

#df = pd.DataFrame({'pred':preds})
#sensor = "Hip"
#df.to_csv(sensor+'_'+'predictions.csv')

## Wrist

In [ ]:
# Run single sensor neural network for detailed activity labels
activity = 'updated_final_activity'
losses = []
accs = []
preds = []
s = "Wrist"
data = clean_data(s, activity, True)
#res = run_nn(data, s, activity)
#sum_cnf_matrix = np.sum(res[2], axis = 0)
acc, cnf_mat = run_nn(data, "Wrist", activity, post = False)
print("---------------------------------------------")
print("Accuracy: ",acc)
print_confusion_matrix(cnf_mat, ['sit/lie', 'stand and move', 'walking', 'running','bicycling'])

#pred = pd.DataFrame(pred)

#pred.to_csv(''+s+'_predictions.csv')

#df = pd.DataFrame({'pred':preds})
#sensor = "Hip"
#df.to_csv(sensor+'_'+'predictions.csv')

## Thigh 

In [ ]:
# Run single sensor neural network for detailed activity labels
activity = 'updated_final_activity'
losses = []
accs = []
preds = []
s = "Thigh"
data = clean_data(s, activity,True)
#res = run_nn(data, s, activity)
#sum_cnf_matrix = np.sum(res[2], axis = 0)

acc, cnf_mat = run_nn(data, "Thigh", activity, post = False)
print("---------------------------------------------")
print("Accuracy: ",acc)
print_confusion_matrix(cnf_mat, ['sit/lie', 'stand and move', 'walking', 'running','bicycling'])

#pred = pd.DataFrame(pred)

#pred.to_csv(''+s+'_predictions.csv')

#df = pd.DataFrame({'pred':preds})
#sensor = "Hip"
#df.to_csv(sensor+'_'+'predictions.csv')

In [ ]:
print_confusion_matrix(cnf_mat, ['sit/lie', 'stand and move', 'walking', 'running','bicycling'])

In [ ]:
x = [[1.43116e+05, 2.43170e+04, 1.38000e+02, 2.00000e+00, 1.14000e+02],
 [2.55780e+04, 9.64500e+04, 5.27300e+03, 2.29000e+02, 2.44000e+02],
 [2.70000e+01, 6.00600e+03, 2.42410e+04, 9.20000e+01, 1.00000e+01],
 [5.90000e+01, 4.20000e+01, 7.56000e+02, 8.32900e+03, 1.20000e+01],
 [1.49000e+02, 5.74000e+03, 2.18000e+02, 1.00000e+00, 0.00000e+00]]

# Functions

- `clean_data` deals with importing the dataset and computing the segemnts
- `run_nn` prepares some features and runs the neural network
    - `run_kfold` is called within this function, it runs the stratified cross-validation and computes the averaged accuracy and other metrics
- `get_model_1layer_updated` is an example of the place to put any varying modeling architecture, to change the model just make a new function and edit the line in `run_kfold` that goes `model = get_model_1layer_updated()` with your new model
- `print_confusion_matrix` displays a confusion matrix
- `post_process` the prediction post processing implementation

In [ ]:
def clean_data(sensor, activity, halved = False):
    """
    Imports the data, adds and drops appropriate columns, and adds segementation to the dataset
    """
    data = pd.read_csv('../../mnt/storage/Datasets/final_data/'+sensor + ".csv", low_memory = False)
    # Take out unnamed columns
    if halved:
        data_halved = data[data.updated_final_activity != 'sit/lie']
        data2 = data[data.updated_final_activity == 'sit/lie'].sample(frac = 0.5)
        data = data_halved.append(data2)
    
    data.drop([col for col in data.columns if "Unnamed" in col], axis=1, inplace = True)
    
    # Drop un-encoded data
    data = data[data[activity] != 'private/not coded'].reset_index()
    # add segment activity
        
    data["segment"] = data["primary_behavior"].map(segments)
    print(data.shape)
    
    # enumerate segments
    # any time the primary behavior changes, we are in a new segment
    number = 0
    segment_group = []
    for index, row in data.iterrows():
        #print(index)
        if index == 0:
            segment_group.append(number)
            #segment_type.append(hip.loc[index,"segment"])
        else:
            current_segment = data.loc[index,"segment"]
            last_segment = data.loc[index - 1,"segment"]
            if last_segment != current_segment:
                # we are in a new segment
                number += 1
            segment_group.append(number)
    data["enum_segment"] = segment_group
    return data

In [ ]:
def run_nn(data, sensor, activity, post = False):
    """
    data: cleaned dataset
    sensor: which sensor we are analyzing (e.g. Hip)
    activity: column name of where the ground truth is
    post: whether to apply post-processing smoothing to the predicted values or not
    """
    x_cols = data.columns[19:(82 + 20)]
    #print(x_cols)
    y = data[activity]
    #print(y.unique())
    # Standardize x columns
    X = data[x_cols].apply(lambda x: (x - np.mean(x)) / np.std(x))
    X = X.fillna(X.mean()).values
    X = pd.DataFrame(X)
    # add segementation and enumeration
    X["segment"] = data.loc[:,"segment"]
    X["enum_segment"] = data.loc[:,"enum_segment"]
    
    # One hot vectorize categories
    Y = pd.get_dummies(y).values
    dummy_labels = pd.get_dummies(y).columns.tolist()
    
    # Run k-fold cross validation
    res = run_kfold(X, Y, 5, sensor, activity, dummy_labels, post_process, True)
    
    # Save confusion matrix visualizations
    #conf_mat(res, sensor, activity, dummy_labels)
    # Calculate precision/recall metrics, save results
    #prec_recall(res, sensor, activity, dummy_labels)
    return res

In [ ]:
def run_kfold(X, Y, K, sensor, activity, class_labels, post,rnn = True):
    acc_sum = 0
    loss_sum = 0
    prec = []
    cnf_tables = []
    cnf_total = np.zeros((5,5))
    f1_rep = []
    prec_rep = []
    rec_rep = []
    i = 1
    model = None
    overall_categories = list(X["segment"].unique())
    for k in range(K):
        # first, select test train split via stratified K-fold cross-validation
        # that is, within each category, do K-fold
        train = []
        test = []
        for cat in overall_categories:
            # extract all rows with this segement and get their enumerated segment
            subset = X[X["segment"] == cat]
            indexes = list(subset["enum_segment"].unique())

            # then stratified train test split
            train += [x for i,x in enumerate(indexes) if i % K != k]
            test += [x for i,x in enumerate(indexes) if i % K == k]
        
        # Reset model and get new one
        model = get_model_1layer_updated()

        # Split data into train and test
        X_train, X_test = X[X["enum_segment"].isin(train)], X[X["enum_segment"].isin(test)]
        y_train, y_test = Y[X_train.index], Y[X_test.index]
            
        # drop segment related columns
        X_train.drop(columns = ["enum_segment","segment"], inplace = True)
        X_test.drop(columns = ["enum_segment","segment"], inplace = True)
        
        if rnn:
            X_train,y_train = aggregate(np.array(X_train),np.array(y_train),7)
            X_test, y_test = aggregate(np.array(X_test),np.array(y_test),7)
            model = get_rnn(X_train)
        # fit the model
        model.fit(X_train, y_train,batch_size=128, epochs= NUM_EPOCHS,
                  verbose = False)
        
        # Confusion matrix calculations
        
        # Get model predictions, save as list
        if rnn:
            reshaped = y_test.shape[0]*y_test.shape[1]
            pred = model.predict(X_test, verbose = 1)
            pred = pred.reshape(reshaped,5)
            y_pred = np.argmax(pred, axis=1)
            y_pred = [class_labels[x] for x in y_pred.tolist()]
        if not rnn:
            pred = model.predict(X_test, verbose = 1)
            y_pred = np.argmax(pred, axis=1)
            y_pred = [class_labels[x] for x in y_pred.tolist()]
        # post-process if necessary
        if post:
            y_pred = post_process(y_pred)
        
        if rnn:
            reshaped = y_test.shape[0]*y_test.shape[1]
            y_true = np.argmax(y_test.reshape(reshaped,5), axis = 1)
            y_true = [class_labels[x] for x in y_true.tolist()]
        # Get ground truth
        if not rnn:
            y_true = np.argmax(y_test, axis = 1)
            y_true = [class_labels[x] for x in y_true.tolist()]
        
        print(y_pred[0:20])
        print(y_true[0:20])
        
        # Evaluate accuracy of model
        acc = sklearn.metrics.accuracy_score(y_true,y_pred)
    
        acc_sum = acc_sum + acc
        i += 1

        # Make confusion matrix
        cnf_matrix = sklearn.metrics.confusion_matrix(y_true, y_pred,
            labels=['sit/lie', 'stand and move', 'walking', 'running',
               'bicycling'])

        # Add confusion matrix for this fold to list
        cnf_tables.append(cnf_matrix)
        
        # Add to total confusion matrix
        cnf_total += cnf_matrix
        
        class_report = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred, 
            labels = ['sit/lie', 'stand and move', 'walking', 'running','bicycling'])
        
        # get f1 for this fold 
        f1_rep.append(class_report[2:4])
        
        # get precision for this fold
        prec_rep.append(itemgetter(0,3)(class_report))
        
        # get recall for this fold
        rec_rep.append(itemgetter(1,3)(class_report))
        

    # Save last trained model to disk
    
    # Get average loss and accuracy for all kfolds
    acc_r = round(acc_sum / K, 3)
    
    # print out weighted average f1-score
    actual_f1 = {'sit/lie':0, 'stand and move':0,'walking':0,'running':0,'bicycling':0}
    total_f1 = {'sit/lie':0,'stand and move':0, 'walking':0,'running':0,'bicycling':0}
    

    for f1 in f1_rep:
        actual_f1['sit/lie'] += f1[0][0] * f1[1][0]
        actual_f1['stand and move'] += f1[0][1] * f1[1][1]
        actual_f1["walking"] += f1[0][2] * f1[1][2]
        actual_f1["running"] += f1[0][3] * f1[1][3]
        actual_f1["bicycling"] += f1[0][4] * f1[1][4]
        total_f1['sit/lie'] += f1[1][0]
        total_f1['stand and move'] += f1[1][1]
        total_f1["walking"] += f1[1][2]
        total_f1["running"] += f1[1][3]
        total_f1["bicycling"] += f1[1][4]
        
    for k in list(actual_f1.keys()):
        print("F1 ",k,":",actual_f1[k]/total_f1[k])
    
    # print out precision stats
    actual_prec = {'sit/lie':0, 'stand and move':0,'walking':0,'running':0,'bicycling':0}
    total_prec = {'sit/lie':0, 'stand and move':0,'walking':0,'running':0,'bicycling':0}
    
    for prec in prec_rep:
        actual_prec['sit/lie'] += prec[0][0] * prec[1][0]
        actual_prec['stand and move'] += prec[0][1] * prec[1][1]
        actual_prec["walking"] += prec[0][2] * prec[1][2]
        actual_prec["running"] += prec[0][3] * prec[1][3]
        actual_prec["bicycling"] += prec[0][4] * prec[1][4]
        total_prec['sit/lie'] += prec[1][0]
        total_prec['stand and move'] += prec[1][1]
        total_prec["walking"] += prec[1][2]
        total_prec["running"] += prec[1][3]
        total_prec["bicycling"] += prec[1][4]
        
    for k in list(actual_prec.keys()):
        print("Precision ",k,":",actual_prec[k]/total_prec[k])
    
    # print out recall stats
    actual_re = {'sit/lie':0, 'stand and move':0,'walking':0,'running':0,'bicycling':0}
    total_re = {'sit/lie':0, 'stand and move':0,'walking':0,'running':0,'bicycling':0}
    
    for rec in rec_rep:
        actual_re['sit/lie'] += rec[0][0] * rec[1][0]
        actual_re['stand and move'] += rec[0][1] * rec[1][1]
        actual_re["walking"] += rec[0][2] * rec[1][2]
        actual_re["running"] += rec[0][3] * rec[1][3]
        actual_re["bicycling"] += rec[0][4] * rec[1][4]
        total_re['sit/lie'] += rec[1][0]
        total_re['stand and move'] += rec[1][1]
        total_re["walking"] += rec[1][2]
        total_re["running"] += rec[1][3]
        total_re["bicycling"] += rec[1][4]
        
    for k in list(actual_re.keys()):
        print("Recall ",k,":",actual_re[k]/total_re[k])
    
    return acc_r, cnf_total

In [ ]:
def aggregate(X,Y,step = 5):
    agged = []
    agged_y = []
    add = (len(X) - len(X) % step) 
    #print(add)
    
    X = X[:add,]
    
    Y = Y[:add,]
    
    #print(X[0:135,])
    i=0
    while i < len(X):
        #print(i)
        x = X[i:i+step,]
        y = Y[i:i+step,]
        #print(x.shape)
        i+= step
        if x.shape != (step,83):
            print('yes',x)
        agged.append(x)
        agged_y.append(y)
    agged = np.array(agged)
    agged_y = np.array(agged_y)
    
    return agged, agged_y

In [ ]:
def get_rnn(X_train):
    '''builds rnn and returns model'''
    model = Sequential()
    model.add(LSTM(128,input_shape = (X_train.shape[1],X_train.shape[2]), return_sequences = True))
    #model.add(Dropout(0.5))
    model.add(LSTM(64, return_sequences = True))
    #model.add(Dropout(0.2))
    model.add(LSTM(32, return_sequences = True))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    #model.add(Dropout(0.4))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    print(model.summary())
    return model

In [ ]:
def get_model_1layer_updated():
    # Neural Network Architecture for single sensor
    model = tf.keras.Sequential([
            layers.Dense(83, activation="relu", input_shape = (83,)),
            layers.Dense(64, activation="relu"),
            layers.Dense(32, activation="relu"),
            layers.Dense(16, activation="relu"),
            layers.Dense(8, activation="relu"),
            layers.Dense(5, activation = 'softmax')])
    model.compile(optimizer="sgd", loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    #fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def post_process(preds):   
    """
    basic post processing...
    prediction: array of predicted values from the NN
    """
    #preds = prediction['predicted_values']
    preds_new = preds.copy()


    for i in range(2, len(preds)-2):
#         size = mode_size*2 +1
#         points = [None]* size
#         for j in range(size):
#             print(j)
#             print(points)
#             points[j] = preds.iloc[i - mode_size]
#             mode_size +=1
        points = [preds[i-2],preds[i-1], preds[i], preds[i+1], preds[i+2]]
        preds_new[i] = stats.mode(points)[0][0]

    #preds_new = pd.DataFrame(preds_new)
    #preds_new.columns = ['post_predictions']
    #final_preds = pd.concat([prediction,preds_new],axis =1)

    #errors_1 = final_preds['y_true'] == final_preds['predicted_values']
    #errors_2 = final_preds['y_true'] == final_preds['post_predictions']

    #print('predicted_value errors',np.sum(errors_1)/len(errors_1))

    #print('post_processed errors',np.sum(errors_2)/len(errors_2))
    
    #print('Accuracy improvement:', (np.sum(errors_2)/len(errors_2)-np.sum(errors_1)/len(errors_1)))
    return preds_new

In [ ]:

#Histograms
sensors = ["Hip","Chest","Wrist", "Thigh"]
#data = pd.read_csv('../../mnt/storage/Datasets/final_data/'+"Hip" + ".csv", low_memory = False)
#print(data.updated_final_activity.unique())


data = clean_data("Hip",'updated_final_activity',True)
sit = len(data[data.updated_final_activity == 'sit/lie'])
stand = len(data[data.updated_final_activity == 'stand and move']) 
walk = len(data[data.updated_final_activity == 'walking']) 
run = len(data[data.updated_final_activity == 'running'])
bike = len(data[data.updated_final_activity == 'bicycling'])
print([sit,stand,walk,run,bike])
x = np.arange(5)
plt.bar(x,height = [sit,stand,walk,run,bike])
plt.xticks(x, ['sit/lie', 'stand and move', 'walking', 'running',
'bicycling'])

In [ ]:
data[data.updated_final_activity != 'sit/lie']